In [ ]:
import pandas as pd

In [ ]:
year = 2022

In [ ]:
qrels_file = f"../data/external/qrels{year}.txt"

In [ ]:
df = pd.read_csv(qrels_file, sep=' ', names=['topic', 'skip','trial', 'qrel'])

In [ ]:
for qrel in [0, 1, 2]:
    print(qrel, len(df[df['qrel'] == qrel]))

In [ ]:
print(f"unique trials: {len(df['trial'].unique())}")

In [ ]:
# trials which appear in the most number of patients
df['trial'].value_counts()

In [ ]:
threshold = len(df['topic'].unique()) - 5
# trials which appear in the most number of patients AND are relevant or excluded
df[(df['trial'].isin(df['trial'].value_counts()[df['trial'].value_counts() > threshold].index)) & df['qrel'].isin([1,2])]['trial'].value_counts()

In [ ]:
eligibile_excluded_stats_df = df[df['qrel'].isin([0,1,2])].groupby(['topic', 'qrel'])['trial'].count()
eligibile_excluded_stats_df

In [ ]:
patients_jsonl = f"../data/external/topics{year}.jsonl"

In [ ]:
import json
with open(patients_jsonl, 'r') as f:
    patients = [json.loads(x) for x in f.readlines()]

In [ ]:
clustering = pd.read_csv(f"../data/external/topics_clustering-2021.csv")
clustering.head()

In [ ]:
for patient_id, patient in enumerate(patients):
    # print(patient_id, patient['patient_id'])
    patients[patient_id]['judged relevant trials'] = int(eligibile_excluded_stats_df[patient['patient_id'], 2])
    patients[patient_id]['judged excluded trials'] = int(eligibile_excluded_stats_df[patient['patient_id'], 1])
    patients[patient_id]['irrelevant trials'] = int(eligibile_excluded_stats_df[patient['patient_id'], 0])

    for index_id, trial in enumerate(df[(df['topic'] == patient['patient_id']) & (df['qrel'] == 2)]['trial'].tolist()[:2]):
        patients[patient_id][f'included {index_id}'] = f"https://clinicaltrials.gov/ct2/show/{trial}"

    for index_id, trial in enumerate(df[(df['topic'] == patient['patient_id']) & (df['qrel'] == 1)]['trial'].tolist()[:1]):
        patients[patient_id][f'excluded {index_id}'] = f"https://clinicaltrials.gov/ct2/show/{trial}"

    del patients[patient_id]['current_medical_history']
    del patients[patient_id]['past_medical_history']
    del patients[patient_id]['family_medical_history']
    del patients[patient_id]['cmh_entities']
    del patients[patient_id]['pmh_entities']
    del patients[patient_id]['fh_entities']

    if year == 2021:
        patients[patient_id]['patient area or specialty'] = clustering[clustering['Disease area or specialty'] == patient['patient_id']]['Topic'].tolist()[0]

In [ ]:
patients

In [ ]:
# save to json, not jsonl
with open(f"../data/external/topics{year}-for-annotation.json", 'w') as f:
    json.dump(patients, f)


In [ ]:
with open(f"../data/external/topics{year}_stats.jsonl", 'w') as f:
    for patient in patients:
        f.write(json.dumps(patient) + '\n')

# stats for patients

In [ ]:
from CTnlp.patient import load_patients_from_xml
import numpy as np

In [ ]:
p21 = load_patients_from_xml("../data/external/topics2021.xml")
p22 = load_patients_from_xml("../data/external/topics2022.xml")

In [ ]:
np.mean([len(x.description.split('.')) for x in p21])

In [ ]:
np.mean([len(x.description.split('.')) for x in p22])

In [ ]:
np.mean([len(x.description.split(' ')) for x in p21])

In [ ]:
np.mean([len(x.description.split(' ')) for x in p22])